In this notebook, we will use Google Vision API to extract text from the images from the Heidelberg Collection in our DFKV Dataset.

In [31]:
import io
import os
import glob
import pandas as pd
# Imports the Google Cloud client library
from google.cloud import vision
from tqdm import tqdm
from IPython.display import Image
from collections import OrderedDict
import math
import json
import shutil
import re
import langid
# Importing Image class from PIL module
from PIL import Image

In [5]:
# Set the environment variable with your own secret key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="absolute/path/to/your/key.json"

In [82]:
def resize_image(url):
    # Opens a image in RGB mode
    im = Image.open(url)

    # Size of the image in pixels (size of original image)
    width, height = im.size

    # Cropped image of above dimension
    # (It will not change original image)
    im1 = im.crop((0, 0, width, 19 * height / 20))

    # Shows the image in image viewer
    im1.save(url)

In [85]:
heidelberg_pages = sorted(glob.glob("./data/heidelberg_data/*.jpg"))

In [84]:
# crop from all the images the bottom, which contains its url
for page in tqdm(heidelberg_pages):
    resize_image(page)

100%|██████████| 5122/5122 [09:02<00:00,  9.44it/s]


In [88]:
def detect_text(path):
    """Detects text in the file."""
    client = vision.ImageAnnotatorClient()

    # Read the image
    with io.open(path, 'rb') as image_file:
        content = image_file.read()
    
    # OCR
    image = vision.Image(content=content)
    response = client.document_text_detection(image=image)
    texts = response.text_annotations

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    #return doc_id and text
    return path.split("/")[3].split("_")[1], path.split("/")[3].split("_")[2], texts[0].description 

In [59]:
# OCR all the documents and save them in a dictionnary
all_texts_heidelberg = dict()
all_texts_pages_heidelberg = dict()
for page in tqdm(heidelberg_pages[:3]):
    doc_id, page, text = detect_text(page)
    #all_texts_heidelberg[doc_id] = all_texts_heidelberg.get(doc_id, "") + text
    all_texts_pages_heidelberg[doc_id + "_" + page] = text

# If there are multiple pages in the same data entry, concatenate them all 
for k, v in all_texts_pages_heidelberg.items():
    all_texts_heidelberg[k.split("_")[0]] = all_texts_heidelberg.get(k.split("_")[0], "") + v

NameError: name 'heidelberg_pages' is not defined

In [97]:
# Save the dictionnary in a json file
with open('./data/complete_texts_heidelberg.json', 'w') as fp:
    json.dump(all_texts_heidelberg, fp)

## Cleaning and tokenize

We will now clean the obtained texts.

In [7]:
# read file
with open('./data/complete_texts_heidelberg.json', 'r') as fp:
    data = json.load(fp)

In [57]:
# Classify between French and German texts
german = dict()
french = dict()

for k, t in tqdm(data.items()):
    language = langid.classify(t.replace("\n", " "))
    if language[0] == "fr":
        french[k] = t.replace("\n", " ")
    else:
        german[k] = t.replace("\n", " ")

100%|██████████| 998/998 [00:08<00:00, 113.13it/s]


In [56]:
langid.classify(french["15413"])

('fr', -4547.86843585968)

In [58]:
french

{'11418': 'Claude Monet 1024 Le Havre. Gartenterrasse am Meer. 1866 Photo: Durand-Ruel / Leinw. 0,98×1,30 m',
 '15413': "nnda est mecessлRun (Dessin de Frédéric Overbeck.) UNE SEULE CHOSE EST NÉCESSAIRE.PRUD'HON SA VIE, SES OEUVRES ET SA CORRESPONDANCE ¹ XI. CES quelques dessins mal rétri- bués ne suffisaient pas à nourrir la famille déjà nombreuse de Pru- d'hon. La misère, que les gaspillages et les criailleries de sa femme ren- daient plus intolérable et plus pro- fonde était à la maison. Ce n'est pas le courage qui manquait au vaillant artiste, mais l'occasion d'exercer son talent. Il n'était pas hostile, tant s'en faut, aux idées et aux institutions nouvelles; cepen- dant il se trouvait absolument isolé. HH.68. J.R. DEL.. Sa manière, qui tenait au fond même de son organisation, était en contradiction flagrante avec celle de David et de son école, et cet homme d'un génie si souple, d'un esprit si ouvert, si conciliant, si sympathique, ne put jamais prendre sur lui de faire la moindr

In [16]:
# List of charachters to remove from texts
to_remove = ["'", ",", ";", ":", ".", "!", "?", "’", "(", ")", "\"", "%", "#", "$", "&", "*", "+"]

In [21]:
t = data["10076"].replace("\n", " ")
t

'147 glitzernde Schlange dienen ihm zum Gegenstand des Studiums. Er entkleidet das Ewig-Weibliche fast ganz seiner geschlechtlichen Reize als Gattung und sucht nur das Einzelwesen. Nur die Sphinx zieht ihn magisch an: kalt, stark und herbe (wie die Büste ,,Das herrische Weib"), oder eine Schattirung zarter und elegischer, aber immer noch sehr herbe und streng im Typus (wie das Relief mit dem lebensgroßen Profil und der Inschrift ,,Auld lang syne"). Nicht ohne Grund mag hier das nordische Idiom gewählt sein: in Schottland und im Norden Englands sind solche Mädchen am häufigsten zu finden. Sie scheinen wie vom Glück enterbte Wesen, die, sich selbst ein Geheimnis, kaum in der Gegenwart leben, in stummer, verzehrender Sehnsucht ihr Dasein verschmachten, stolz und kalt nach außen, aber ewig suchend nach dem großen Unerklärlichen. So steht dieses Mädchen mit den mageren entblößten Armen am Sarkophag und träumt von der Vergangenheit und Zu- kunft und vom Glück, das niemals kommt. Ent- behren 

In [30]:
re.search(r'http', t).group()

'http'

In [33]:
o = langid.classify(t)

In [34]:
o[0]

'de'